In [ ]:
import os

OPENAI_API_KEY=os.environ['OPENAI_API_KEY'] 

In [2]:
!pip install langchain openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00


In [3]:
from langchain.llms import OpenAI

In [4]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.9)

In [5]:
llm("What is LLM")

'\n\nLLM stands for "Legum Magister" and is an advanced postgraduate academic degree in law. It is typically a one-year program for those who have already completed a first law degree at an accredited institution. The LLM program usually includes a combination of academic coursework and a research paper or thesis, focusing on a specific area of law. In the US, LLM programs are offered by many top law schools and the degree opens up career opportunities in the field of law.'

In [6]:
from langchain.schema import AIMessage,HumanMessage,SystemMessage
from langchain.chat_models import ChatOpenAI

In [7]:
chat = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0.3,openai_api_key=OPENAI_API_KEY)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content = "Write a Python script that trains a neural network on simulated data ")
]

In [8]:
response = chat(messages)

In [9]:
print(response.content, end='\n')

Sure! Here's a simple example of a Python script that trains a neural network on simulated data using the Keras library:

```python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Generate simulated data
np.random.seed(0)
X = np.random.rand(100, 2)
y = np.random.randint(2, size=(100, 1))

# Define the neural network model
model = Sequential()
model.add(Dense(32, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=10)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")
```

In this script, we first generate simulated data using `numpy.random.rand` and `numpy.random.randint`. We then define a simple neural network model using the `Sequential` class from Keras and add two dense layers with ReLU and sigmoid a

## PROMPTS IN LANGCHAIN

In [10]:
from langchain import PromptTemplate

In [11]:
template = """
You are an expert data scientist with an expertise in building deep learning models.
Explain the concept of {concept} in a couple of lines
"""

In [12]:
prompt = PromptTemplate(
    input_variables=["concept"],
    template = template
)

In [13]:
llm(prompt.format(concept= "regularization"))

'\nRegularization is a technique used to improve the generalization of a model and reduce the likelihood of model overfitting by regularizing (i.e., adding constraints to) the weights and biases of a model. This is typically done by introducing a penalty term to the loss function; most commonly a L1 or L2 penalty.'

## CHAINs

In [15]:
from langchain.chains import LLMChain

In [16]:
chain = LLMChain(llm = llm, prompt = prompt)

In [17]:
print(chain.run("Autoencoder"))


Autoencoders are a type of unsupervised deep learning models used to learn efficient representations of data by compressing them and reconstructing the data from the compressed version. They are used for feature extraction and dimensionality reduction.


#### Sequential chain of multiple chains

In [18]:
second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template = "Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words"
)
chain_two = LLMChain(llm = llm, prompt = second_prompt)

In [19]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains = [chain, chain_two], verbose = True)

In [20]:
explaination = overall_chain.run("Autoencoders")
print(explaination)



> Entering new SimpleSequentialChain chain...

Autoencoders are unsupervised neural networks that are used to learn data representations through compression and reconstruction, by mapping the input to a low dimensional representation based on hidden layers (encoders), and then decoding it back to the original domain (decoders). They are used to identify patterns and detect outliers in data.


Autoencoders are special kinds of computer programs that help us to learn about data. They are like a game of hide and seek, where the computer has to figure out how to store and remember data.

To play the game, the autoencoder has a few important steps. The first step is called the encoder. This is where the data is put into a special box and is compressed into a much smaller size. It's like a magician shrinking an elephant into a suitcase! The shrunken data is then sent to a hidden layer (like a secret hideout) where it is stored. 

The next step is called the decoder. This is where the data 

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 0
)

In [25]:
texts = text_splitter.create_documents([explaination])

In [26]:
texts

[Document(page_content='Autoencoders are special kinds of computer programs that help us to learn about data. They are like a game of hide and seek, where the computer has to figure out how to store and remember data.', metadata={}),
 Document(page_content="To play the game, the autoencoder has a few important steps. The first step is called the encoder. This is where the data is put into a special box and is compressed into a much smaller size. It's", metadata={}),
 Document(page_content='like a magician shrinking an elephant into a suitcase! The shrunken data is then sent to a hidden layer (like a secret hideout) where it is stored.', metadata={}),
 Document(page_content="The next step is called the decoder. This is where the data is taken out of the hideout and made back to its original size. It's like a magician pulling an elephant out of a suitcase! The data is", metadata={}),
 Document(page_content='then sent back to us in its original form.', metadata={}),
 Document(page_content